In [ ]:
import numpy

# Intialise OpenCMISS-Iron.
from opencmiss.iron import iron

In [ ]:
# Create coordinate system.
coordinate_system_user_number = 1
coordinate_system = iron.CoordinateSystem()
coordinate_system.CreateStart(coordinate_system_user_number)
coordinate_system.DimensionSet(3)
coordinate_system.CreateFinish()

In [ ]:
# File name
file_name = 'gmsh2iron'

#Inputing element file
elem_file = open(file_name + '.ele', 'r')

ele_file_list = list()
element_nodes = list()
for i, line in enumerate(elem_file):
    ele_file_list.append(line)
    if ele_file_list[i].split()[0] == "tetrahedron":
        element_nodes.append(ele_file_list[i].split()[3])
        element_nodes.append(ele_file_list[i].split()[4])
        element_nodes.append(ele_file_list[i].split()[5])
        element_nodes.append(ele_file_list[i].split()[6])
    else:
        continue
        
elem_nodes_used = set(element_nodes)

# Reading node file
node_file = open(file_name + '.node', 'r')

node_file_list = list()
for i, line in enumerate(node_file):
    node_file_list.append(line)
    if (node_file_list[-1][:].split()[0] in elem_nodes_used) or (i == 0):
        continue
    else: 
        node_file_list.pop()

In [ ]:
# As formatted to gmsh documentation
# numEntityBlocks(size_t) numNodes(size_t) minNodeTag(size_t) maxNodeTag(size_t)

# Reading the mesh details from the first line of the node file
numEntityBlocks, numNodes, minNodeTag, maxNodeTag = node_file_list[0].split()
num_nodes = len(elem_nodes_used)
num_coords= 3
num_attributes=0
boundary_markers=0
# Creating variables to store node number & boundary marker
NodeNums = [[0 for m in range(2)] for n in range(num_nodes)]
# Creating array to store x and y coordinates
NodeCoords = [[0 for m in range(num_coords)] for n in range(num_nodes)]

nodeIDs = list()
# Reading details from node fileb
for i in range(num_nodes):
    NodeNums[i][0], NodeCoords[i][0], NodeCoords[i][1], NodeCoords[i][2] = node_file_list[i+1].split()  #node number, x, y, z, boundary marker
    # Converting from string to appropriate type
    NodeNums[i][0] = int(NodeNums[i][0])
    NodeCoords[i][0] = float(NodeCoords[i][0])
    NodeCoords[i][1] = float(NodeCoords[i][1])
    NodeCoords[i][2] = float(NodeCoords[i][2])
    nodeIDs.append(NodeNums[i][0])

# Closing the file
node_file.close()

In [ ]:
# gmsh .msh file format:
# numEntityBlocks(size_t) numElements(size_t) minElementTag(size_t) maxElementTag(size_t)

#Reading the values of the first line
numEntityBlocks, numElements, minElementTag, maxElementTag = ele_file_list[0].split()

# {1: "line", 2: "triangle", 3: "quadrangle", 4: "tetrahedron", 5: "hexahedron", 6: "prism", 7: "pyramid", 15: "point"}
types = {4: "tetrahedron", 5: "hexahedron", 6: "prism", 7: "pyramid"}

element_type = list()
count = 0
for i, line in enumerate(ele_file_list):
    if line.split()[0] in types.values():
        element_type.append(line.split()[0])

# element_count.append(count)
element_type_count = list()
element_type_set = set(element_type)

element_type_count = map(lambda x: element_type.count(x), element_type_set)

elements_breakdown = dict(zip(element_type_set, list(element_type_count)))
print(elements_breakdown)

ele_attributes = 0

# Condition on the different amount of mesh elements (3 dimensional)
if len(elements_breakdown.values()) >= 3:
    # element counts
    num_ele_tetra = elements_breakdown["tetrahedron"]
    num_ele_quadr = elements_breakdown["quadrangle"]
    num_ele_pyram = elements_breakdown["pyramid"]
    # node counts
    num_nodes_tetra = 4
    num_nodes_quadr = 4
    num_nodes_pyram = 5
    # Creating variable to store the element map
    ele_map_tetra = [[0 for x in range(num_nodes_tetra + ele_attributes)] for y in range(num_ele_tetra)]
    ele_map_quadr = [[0 for x in range(num_nodes_quadr + ele_attributes)] for y in range(num_ele_quadr)]
    ele_map_pyram = [[0 for x in range(num_nodes_pyram + ele_attributes)] for y in range(num_ele_pyram)]
    # Element indexes
    ele_idx_tetra = [[0 for m in range(1)] for n in range(num_ele_tetra)]
    ele_idx_quadr = [[0 for m in range(1)] for n in range(num_ele_quadr)]
    ele_idx_pyram = [[0 for m in range(1)] for n in range(num_ele_pyram)]
    # Reading element data from elemfile
    ele_idx_count_tetra = 0
    ele_idx_count_quadr = 0
    ele_idx_count_pyram = 0
    check = 0
    for line in ele_file_list:
        if line.split()[0] == "tetrahedron":
            _, _, _, ele_map_tetra[ele_idx_count_tetra][0], ele_map_tetra[ele_idx_count_tetra][1], ele_map_tetra[ele_idx_count_tetra][2], ele_map_tetra[ele_idx_count_tetra][3] = line.split()
            ele_idx_count_tetra += 1
            ele_idx_tetra[ele_idx_count_tetra-1][0] = ele_idx_count_tetra
        elif line.split()[0] == "quadrangle":
            _, _, _, ele_map_quadr[ele_idx_count_quadr][0], ele_map_quadr[ele_idx_count_quadr][1], ele_map_quadr[ele_idx_count_quadr][2], ele_map_quadr[ele_idx_count_quadr][3] = line.split()
            ele_idx_count_quadr += 1
            ele_idx_quadr[ele_idx_count_quadr-1][0] = ele_idx_count_quadr
        elif line.split()[0] == "pyramid":
            _, _, _, ele_map_pyram[ele_idx_count_pyram][0], ele_map_pyram[ele_idx_count_pyram][1], ele_map_pyram[ele_idx_count_pyram][2], ele_map_pyram[ele_idx_count_pyram][3], ele_map_pyram[ele_idx_count_pyram][4] = line.split()
            ele_idx_count_pyram += 1
            ele_idx_pyram[ele_idx_count_pyram-1][0] = ele_idx_count_pyram
    
elif len(elements_breakdown.values()) == 2:
    # element counts
    num_ele_tetra = elements_breakdown["tetrahedron"]
    num_ele_quadr = elements_breakdown["pyramid"]
    # node counts
    num_nodes_tetra = 4
    num_nodes_quadr = 4
    # Creating variable to store the element map
    ele_map_tetra = [[0 for x in range(num_nodes_tetra + ele_attributes)] for y in range(num_ele_tetra)]
    ele_map_quadr = [[0 for x in range(num_nodes_quadr + ele_attributes)] for y in range(num_ele_quadr)]
    # Element indexes
    ele_idx_tetra = [[0 for m in range(1)] for n in range(num_ele_tetra)]
    ele_idx_quadr = [[0 for m in range(1)] for n in range(num_ele_quadr)]
    # Reading element data from elemfile
    ele_idx_count_tetra = 0
    ele_idx_count_quadr = 0
    check = 0
    for line in ele_file_list:
        if line.split()[0] == "tetrahedron":
            _, _, _, ele_map_tetra[ele_idx_count_tetra][0], ele_map_tetra[ele_idx_count_tetra][1], ele_map_tetra[ele_idx_count_tetra][2], ele_map_tetra[ele_idx_count_tetra][3] = line.split()
            ele_idx_count_tetra += 1
            ele_idx_tetra[ele_idx_count_tetra-1][0] = ele_idx_count_tetra
        elif line.split()[0] == "quadrangle":
            _, _, _, ele_map_quadr[ele_idx_count_quadr][0], ele_map_quadr[ele_idx_count_quadr][1], ele_map_quadr[ele_idx_count_quadr][2], ele_map_quadr[ele_idx_count_quadr][3] = line.split()
            ele_idx_count_quadr += 1
            ele_idx_quadr[ele_idx_count_quadr-1][0] = ele_idx_count_quadr
            
elif len(elements_breakdown.values()) == 1:
    # element counts
    num_ele_tetra = elements_breakdown["tetrahedron"]
    # node counts
    num_nodes_tetra = 4
    # Creating variable to store the element map
    ele_map_tetra = [[0 for x in range(num_nodes_tetra + ele_attributes)] for y in range(num_ele_tetra)]
    # Element indexes
    ele_idx_tetra = [[0 for m in range(1)] for n in range(num_ele_tetra)]
    # Reading element data from elemfile
    ele_idx_count = 0
    check = 0
    for line in ele_file_list:
        if line.split()[0] == "tetrahedron":
            _, _, _, ele_map_tetra[ele_idx_count][0], ele_map_tetra[ele_idx_count][1], ele_map_tetra[ele_idx_count][2], ele_map_tetra[ele_idx_count][3] = line.split()
            ele_idx_count += 1
            ele_idx_tetra[ele_idx_count-1][0] = ele_idx_count

# # converting 2d list into 1d
# num_tetra_nodes = len(set(chain.from_iterable(ele_map_tetra)))

# Closing the file
elem_file.close()

In [ ]:
numberOfXi = 3
basis_user_number = 1
basis = iron.Basis()
basis.CreateStart(basis_user_number)
basis.type = iron.BasisTypes.SIMPLEX
basis.numberOfXi = numberOfXi
basis.interpolationXi = [iron.BasisInterpolationSpecifications.LINEAR_SIMPLEX] * numberOfXi
basis.quadratureOrder = 2
basis.CreateFinish()

In [ ]:
# Create region
region_user_number = 1
region = iron.Region()
region.CreateStart(region_user_number, iron.WorldRegion)
region.CoordinateSystemSet(coordinate_system)
region.LabelSet("Region")
region.CreateFinish()

In [ ]:
# Initialise Nodes
nodes = iron.Nodes()
nodes.CreateStart(region, num_nodes)
nodes.CreateFinish()

In [ ]:
# Initialise Mesh
mesh = iron.Mesh()
mesh_user_number=1
mesh.CreateStart(mesh_user_number, region, num_coords)
mesh.NumberOfElementsSet(num_ele_tetra)
mesh.NumberOfComponentsSet(1)

In [ ]:
# Initialise Elements
meshElements = iron.MeshElements()
meshElements.CreateStart(mesh, 1, basis)

In [ ]:
for i in range(num_ele_tetra):
    element = ele_idx_tetra[i][0]
    
    try: 
        nodesList = list(
          map(int,[ele_map_tetra[i][0], ele_map_tetra[i][1], ele_map_tetra[i][2], ele_map_tetra[i][3]]))
        meshElements.NodesSet(int(element), nodesList)   
    except:
        print("error at {}".format(i+638))
        print("with nodesList {}".format(nodesList))
        print((nodesList[0] in nodeIDs), (nodesList[1] in nodeIDs),(nodesList[2] in nodeIDs),(nodesList[3] in nodeIDs),)
    
        
meshElements.CreateFinish()

In [ ]:
# Finalise Mesh
mesh.CreateFinish()

In [ ]:
# Perform mesh decomposition.
decomposition_user_number = 1
decomposition = iron.Decomposition()
decomposition.CreateStart(decomposition_user_number, mesh)
decomposition.CreateFinish()

In [ ]:
# Create a field for the geometry.
geometric_field_user_number = 1
geometric_field = iron.Field()
geometric_field.CreateStart(geometric_field_user_number, region) #notice that the geometric field is associated with region in this function call.
geometric_field.LabelSet('Geometry')
geometric_field.MeshDecompositionSet(decomposition)
geometric_field.CreateFinish()

In [ ]:
# Set geometric field values from customized mesh.
computationalNodeNumber = iron.ComputationalNodeNumberGet()
for node_idx in range(num_nodes):
    try:
        node_id = NodeNums[node_idx][0]
        node_x = NodeCoords[node_idx][0]
        node_y = NodeCoords[node_idx][1]
        node_z = NodeCoords[node_idx][2]
        geometric_field.ParameterSetUpdateNodeDP(
          iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES,
          1, 1, node_id, 1, node_x)
        geometric_field.ParameterSetUpdateNodeDP(
          iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES,
          1, 1, node_id, 2, node_y)
        geometric_field.ParameterSetUpdateNodeDP(
          iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES,
          1, 1, node_id, 3, node_z)
    except:
        print("error at node_idx {}".format(node_idx))
        print("with node coordinates {}".format([node_x, node_y, node_z]))

In [ ]:
# Update the geometric field.
geometric_field.ParameterSetUpdateStart(
  iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)
geometric_field.ParameterSetUpdateFinish(
  iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)

## /////////////////

In [ ]:
dependent_field_user_number = 2

dependent_field = iron.Field()
dependent_field.CreateStart(dependent_field_user_number, region)
dependent_field.MeshDecompositionSet(decomposition)
dependent_field.TypeSet(iron.FieldTypes.GEOMETRIC_GENERAL)
dependent_field.GeometricFieldSet(geometric_field)
dependent_field.DependentTypeSet(iron.FieldDependentTypes.DEPENDENT)
dependent_field.VariableLabelSet(iron.FieldVariableTypes.U, "Dependent")
dependent_field.NumberOfVariablesSet(2)

# Set the number of componets for the U variable (position) and the DELUDELN
# (forces).
dependent_field.NumberOfComponentsSet(iron.FieldVariableTypes.U, 4)
dependent_field.NumberOfComponentsSet(iron.FieldVariableTypes.DELUDELN, 4)

if use_pressure_basis:
    # Set the hydrostatic pressure to be nodally based and use the second mesh component.
    # U variable (position)
    dependent_field.ComponentInterpolationSet(
        iron.FieldVariableTypes.U, 4, iron.FieldInterpolationTypes.NODE_BASED)
    dependent_field.ComponentMeshComponentSet(
        iron.FieldVariableTypes.U, 4, 2)

    # DELUDELN variable (forces)
    dependent_field.ComponentInterpolationSet(
        iron.FieldVariableTypes.DELUDELN, 4,
        iron.FieldInterpolationTypes.NODE_BASED)
    dependent_field.ComponentMeshComponentSet(
        iron.FieldVariableTypes.DELUDELN, 4, 2)

    if interpolation_type == 4:
        # Set arc length scaling for cubic-Hermite elements.
        dependent_field.FieldScalingTypeSet(
            iron.FieldScalingTypes.ARITHMETIC_MEAN)
else:
    # Set the hydrostatic pressure to be constant within each element.
    dependent_field.ComponentInterpolationSet(
        iron.FieldVariableTypes.U, 4,
        iron.FieldInterpolationTypes.ELEMENT_BASED)
    dependent_field.ComponentInterpolationSet(
        iron.FieldVariableTypes.DELUDELN, 4,
        iron.FieldInterpolationTypes.ELEMENT_BASED)
dependent_field.CreateFinish()

In [ ]:
# Initialise dependent field from undeformed geometry and displacement bcs and set hydrostatic pressure.
iron.Field.ParametersToFieldParametersComponentCopy(
    geometric_field, iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 1,
    dependent_field, iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 1)
iron.Field.ParametersToFieldParametersComponentCopy(
    geometric_field, iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 2,
    dependent_field, iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 2)
iron.Field.ParametersToFieldParametersComponentCopy(
    geometric_field, iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 3,
    dependent_field, iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 3)
iron.Field.ComponentValuesInitialiseDP(
    dependent_field, iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 4, 0.0)

In [ ]:
# Create the material field.
material_field_user_number = 3

material_field = iron.Field()
material_field.CreateStart(material_field_user_number, region)
material_field.TypeSet(iron.FieldTypes.MATERIAL)
material_field.MeshDecompositionSet(decomposition)
material_field.GeometricFieldSet(geometric_field)
material_field.VariableLabelSet(iron.FieldVariableTypes.U, "Material")

# Set the number of components for the Mooney Rivlin constitutive equation (2).
material_field.NumberOfComponentsSet(iron.FieldVariableTypes.U, 2)

for component in [1, 2]:
    material_field.ComponentInterpolationSet(
      iron.FieldVariableTypes.U, component,
      iron.FieldInterpolationTypes.ELEMENT_BASED)
    if interpolation_type == 4:
        # Set arc length scaling for cubic-Hermite elements.
        material_field.FieldScalingTypeSet(iron.FieldScalingTypes.ARITHMETIC_MEAN)
material_field.CreateFinish()

# Set Mooney-Rivlin constants c10 and c01 respectively.
material_field.ComponentValuesInitialiseDP(
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 1, 1.0)
material_field.ComponentValuesInitialiseDP(
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 2, 0.2)

In [ ]:
# Equation set field.
equations_set_field_user_number = 4
equations_set_field = iron.Field()
equations_set = iron.EquationsSet()